質問文を学習させ、正当な分類を行う

In [2]:
!pip install fugashi ipadic transformers unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=a767d8c984bb783ced9213e86bf98c463e374c19e99c951daeebc906d86f6409
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルDBの用意

In [1]:
import pandas as pd
# データフレームの取得
file_path = "/content/drive/MyDrive/チャットボット/大学/program/提出用/test_question.csv"#"C:/Users/iput Hata Sakuto/cuda-11.8/Univercity/Answer_output/chunk2question_after.xlsx" # <- 「use_df.xlsx」にて、カラムが空になっていたものを削除した
df = pd.read_csv(file_path, encoding='utf-8')
df

,test_question,label_num,label,test_q_embedding,test_chunk
0,東京国際工科専門職大学の使命は何ですか？,0,専門職大学,"[0.0005529869231395423, -0.0019058298785239458...",現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...
1,現代社会の課題はどのような要素で複雑に絡み合っていますか？,0,専門職大学,"[0.0016240725526586175, 0.01832978054881096, 0...",現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...
2,解決するためには、どのような知識が必要ですか？,0,専門職大学,"[0.039757076650857925, 0.006085861008614302, 0...",現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...
3,専門分野の知識だけでは、課題の解決は困難ですか？,0,専門職大学,"[0.019077474251389503, -0.010898500680923462, ...",現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...
4,なぜ私たちは既存の学問の壁を超える必要がありますか？,0,専門職大学,"[0.01970493234694004, -0.0004433440917637199, ...",現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...
...,...,...,...,...,...
1185,深層学習の効果にはどのようなものがありますか？,3,授業,"[-0.021070925518870354, -0.009638827294111252,...",深層学習 深層学習(Deep Learning) の主要技術であるニューラルネットワークの技...
1186,深層学習の講義では具体的な技術を学ぶことができますか？,3,授業,"[-0.010601399466395378, -0.007428060285747051,...",深層学習 深層学習(Deep Learning) の主要技術であるニューラルネットワークの技...
1187,画像認識システムなどで実用される深層学習技術の適用例には具体的な実装例がありますか？,3,授業,"[-0.019572701305150986, 0.002586578717455268, ...",深層学習 深層学習(Deep Learning) の主要技術であるニューラルネットワークの技...
1188,畳み込みニューラルネットワーク（CNN）の構築例題を通じて実装例を修得できますか？,3,授業,"[-0.003475097706541419, 0.01182270422577858, 0...",深層学習 深層学習(Deep Learning) の主要技術であるニューラルネットワークの技...


モデル作成

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('rinna/japanese-gpt2-medium') #cl-tohoku/bert-base-japanese-whole-word-masking')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
import pandas as pd
# 各クラスごとに100個のサンプルを抽出して結合
selected_samples = pd.concat([
    df[df['label_num'] == 0].sample(200, random_state=42),
    df[df['label_num'] == 1].sample(200, random_state=42),
    df[df['label_num'] == 2].sample(200, random_state=42),
    df[df['label_num'] == 3].sample(200, random_state=42)
], axis=0)

# 選択したサンプルをテストデータとして分割
X_train = selected_samples["test_q_embedding"].tolist()
y_train = selected_samples['label']

# 学習データとテストデータ以外のサンプルを訓練データとして使用
X_test = (df["test_q_embedding"].drop(index=selected_samples.index)).tolist()
y_test = df['label'].drop(index=selected_samples.index)

In [4]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
import tensorflow as tf

label2index = {
    '専門職大学': 0,
    '特徴': 1,
    '学び': 2,
    '授業': 3,
}

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.map(lambda x: label2index[x]).to_list()
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test.map(lambda x: label2index[x]).to_list()
))


In [ ]:
from transformers import TFTrainer, TFTrainingArguments, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tensorflow as tf

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TFTrainingArguments(
    output_dir='/content/drive/MyDrive/チャットボット/大学/program/提出用/質問文から質問文の分類を行う予測モデル/result',# output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/チャットボット/大学/program/提出用/質問文から質問文の分類を行う予測モデル/logs',            # directory for storing logs
    logging_steps=10
)

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('rinna/japanese-gpt2-medium') #cl-tohoku/bert-base-japanese-whole-word-masking')
    # Transformers 4.5.1のTFBertForSequenceClassificationでは
    # classifierにactivationが設定されておらず、多項分類でlossが計算出来ないため、
    # 差し替えています。
    model.classifier = tf.keras.layers.Dense(
            units=4, # クラス数を指定します。
            activation='softmax',
            name='classifier',
        )

trainer = TFTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

You are using a model of type gpt2 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['transformer.h.18.ln_2.weight', 'transformer.h.20.attn.c_proj.weight', 'transformer.h.22.mlp.c_fc.bias', 'transformer.h.16.mlp.c_proj.bias', 'transformer.h.3.mlp.c_fc.bias', 'transformer.h.15.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.c_attn.weight', 'transformer.h.8.ln_2.bias', 'transformer.h.23.attn.c_proj.bias', 'transformer.h.23.ln_2.weight', 'transformer.h.11.mlp.c_fc.bias', 'transformer.h.12.attn.c_attn.weight', 'transformer.h.2.mlp.c_proj.weight', 'transformer.h.16.ln_1.bias', 'transformer.h.8.attn.c_proj.weight', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.21.attn.c_attn.weight', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.19.mlp.c_fc.bias', 'transformer.h.19.ln_1.bias', 'transformer.h.23.mlp.c_fc.weight', 'transformer.h.23.ln_1.weight', 'transformer.h.22.mlp.c_fc.weight', 'transformer.h.4.mlp.c_proj.bias', 'transforme